## OpenAI API Key setup

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Load ECB

datasets adopted from [ahmeshaf/lemma_ce_coref](https://github.com/ahmeshaf/lemma_ce_coref)

In [2]:
import pickle

In [3]:
# load the ECB+ dataset
dataset_folder = f'datasets/ecb/'
mention_map = pickle.load(open(dataset_folder + "/mention_map.pkl", 'rb'))
evt_mention_map = {m_id: m for m_id, m in mention_map.items() if m['men_type'] == 'evt'}

In [4]:
# load the predictions made by the lemma heuristics on the \
# dev set of the ECB+ dataset
dev_preidctions = pickle.load(open("datasets/ecb/lh/mp_mp_t_dev.pkl", 'rb'))
_, (tps_trans, fps_trans, tns_trans, fns_trans) = dev_preidctions

In [5]:
tps_trans[0]

('12_11ecbplus.xml_21', '12_10ecbplus.xml_9')

In [6]:
evt_mention_map.get('12_11ecbplus.xml_21')

{'m_id': '21',
 'sentence_id': '3',
 'topic': '12',
 'men_type': 'evt',
 'split': 'dev',
 'mention_text': 'attempt',
 'sentence': 'Warship INS Sukanya on Thursday foiled a piracy attempt in the Gulf of Aden between Somalia and Yemen .',
 'doc_id': '12_11ecbplus.xml',
 'type': 'action_occurrence',
 'start': 8,
 'end': 8,
 'bert_sentence': 'Warship INS Sukanya on Thursday foiled a piracy <m> attempt </m> in the Gulf of Aden between Somalia and Yemen .',
 'bert_doc': 'http : / / www . indianexpress . com / news / ins - sukanya - foils - piracy - attempt - in - gulf - of - eden / 874560 /\nINS Sukanya foils piracy attempt in Gulf of Eden\nSat Nov 12 2011 , 01 : 19 hrs\nWarship INS Sukanya on Thursday foiled a piracy <m> attempt </m> in the Gulf of Aden between Somalia and Yemen .\nThe ship was escorting a group of five merchant vessels through the Internationally Recognised Transit Corridor ( IRTC ) when the bid occurred .\nThis is the fifth anti - piracy operation carried out by the warsh

## Baseline Experiments with Langchain

The output parser may initially appear redundant, given the straightforward nature of the baseline output. <br>
However, its utility becomes useful when implementing advanced prompting methods, such as the `chain of thought` technique.

In [7]:
from langchain import LLMChain, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

__Baseline prompt__

You are en expert Event Coreference Resolution System. <br>
I will provide two sentences with marked events. Your task is to identify if they both refer to the same event.<br>
If they are coreferent, please return "True", otherwise write "False".

In [8]:
baseline_template = """
You are en expert Event Coreference Resolution System. I will provide two sentences with marked events. Your task is to identify if they both refer to the same event. If they are coreferent, please return "True", otherwise write "False".
{format_instructions}
Event1: {event1}
Event2: {event2}
Your answer: 
"""

In [9]:
baseline_response_schemas = [
    ResponseSchema(
        name="Answer",
        description=("Determines if two marked events in provided sentences refer to the same event."
                     "You answer should only be 'True' or 'False'.")
    ),
]
baseline_output_parser = StructuredOutputParser.from_response_schemas(baseline_response_schemas)
baseline_format_instructions = baseline_output_parser.get_format_instructions()

In [10]:
baseline_prompt = PromptTemplate(
    template=baseline_template,
    input_variables=['event1', 'event2'],
    partial_variables={"format_instructions": baseline_format_instructions},
)

In [11]:
llm = ChatOpenAI(temperature=0.0, model='gpt-4')
basline_chain = LLMChain(llm=llm, prompt=baseline_prompt)

In [12]:
# example: a pair of events from true positve set
event1 = evt_mention_map.get(tps_trans[0][0])['bert_sentence']
event2 = evt_mention_map.get(tps_trans[0][1])['bert_sentence']
print(event1)
print(event2)

Warship INS Sukanya on Thursday foiled a piracy <m> attempt </m> in the Gulf of Aden between Somalia and Yemen .
Indian navy has foiled a piracy <m> attempt </m> on 5 Indian merchant vessels in international waters .


In [19]:
predict = basline_chain.run(event1=event1, event2=event2, verbose=True)
predict_dict = baseline_output_parser.parse(predict)
print(predict_dict)

{'Answer': 'False'}


The GPT 4 got this pair wrong...

### Experiment in scope

In [16]:
import os
from collections import defaultdict
from tqdm import tqdm

In [ ]:
tps_trans_baseline_results = defaultdict(dict)

for evt_pair in tqdm(tps_trans):
    event1_id = evt_pair[0]
    event2_id = evt_pair[1]
    
    event1_data = evt_mention_map.get(event1_id)
    event2_data = evt_mention_map.get(event2_id)

    if event1_data is None or event2_data is None:
        continue

    event1 = event1_data['bert_sentence']
    event2 = event2_data['bert_sentence']

    predict = basline_chain.run(event1=event1, event2=event2)
    predict_dict = baseline_output_parser.parse(predict)

    tps_trans_baseline_results[evt_pair] = {
        'event1_id': event1_id,
        'event2_id': event2_id,
        'event1': event1,
        'event2': event2,
        'predict_dict': predict_dict,
        'gold_label': True
    }

In [ ]:
fps_trans_baseline_results = defaultdict(dict)

for evt_pair in tqdm(fps_trans):
    event1_id = evt_pair[0]
    event2_id = evt_pair[1]
    
    event1_data = evt_mention_map.get(event1_id)
    event2_data = evt_mention_map.get(event2_id)

    if event1_data is None or event2_data is None:
        continue

    event1 = event1_data['bert_sentence']
    event2 = event2_data['bert_sentence']

    predict = basline_chain.run(event1=event1, event2=event2)
    predict_dict = baseline_output_parser.parse(predict)

    fps_trans_baseline_results[evt_pair] = {
        'event1_id': event1_id,
        'event2_id': event2_id,
        'event1': event1,
        'event2': event2,
        'predict_dict': predict_dict,
        'gold_label': False
    }


In [ ]:
save_folder = 'datasets/ecb/llm/dev'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

with open(f'{save_folder}/tps_trans_baseline_results.pkl', 'wb') as f:
    pickle.dump(tps_trans_baseline_results, f)

with open(f'{save_folder}/fps_trans_baseline_results.pkl', 'wb') as f:
    pickle.dump(fps_trans_baseline_results, f)

